In [2]:
#-------------------------------------------------------IMPORT----------------------------------------------------------------------------#
#dieu
import numpy as np
import pandas as pd
import string
from collections import defaultdict
from time import time
# Sklearn
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer , TfidfTransformer
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import recall_score , precision_score ,f1_score
from sklearn import metrics , svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, ComplementNB, BernoulliNB
from sklearn.svm import SVC, LinearSVC
#vis
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly as py
from plotly.offline import init_notebook_mode, iplot, plot
#NLP
import nltk
from nltk.corpus import stopwords , wordnet as wn
from nltk import wordpunct_tokenize , WordNetLemmatizer ,sent_tokenize ,  word_tokenize
from nltk.stem import PorterStemmer , LancasterStemmer
from nltk.stem.snowball import SnowballStemmer  

In [3]:
#------------------------------------------------------DATAFRAME DISPLAY------------------------------------------------------------------#
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)
pd.reset_option("^display") ##Reset display option

In [57]:
#-----------------------------------------------------------FUNCTIONS--------------------------------------------------------------------#
def get_top_n_words(corpus,d,n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    if d == "up" :
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]
    elif d == "down" :
        words_freq=sorted(words_freq, key = lambda x: x[1], reverse=False)
        return words_freq[:n]

def tokenize(corpus):
    #text = ''.join([ch for ch in text if ch not in dataEF["Text"]])
    tokens = nltk.word_tokenize()
    lemma = WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    # tokens = [lemmatizer.lemmatize(token) for token in tokens] 
    #OR
    # tokens = [lemma.lemmatize(lemma.lemmatize(lemma.lemmatize(w,'v'),'n'),'a')for w in tokens]
    return [stemmer.stem(token) for token in tokens]

def run_pipes(pipes, splits=10, test_size=0.2, seed=42):  
    res = defaultdict(list)
    spliter = ShuffleSplit(n_splits=splits, test_size=test_size, random_state=seed)
    for idx_train, idx_test in spliter.split(corpus):
        for pipe in pipes:
            # name of the model
            name = "-".join([x[0] for x in pipe.steps])
            
            # extract datasets
            X_train = corpus[idx_train]
            X_test = corpus[idx_test]
            y_train = targets[idx_train]
            y_test = targets[idx_test]
            
            # Learn
            start = time()
            pipe.fit(X_train, y_train)
            fit_time = time() - start
            
            # predict and save results
            y = pipe.predict(X_test)
            res[name].append([
                fit_time,
                f1_score(y_test, y, average = 'micro'),
                precision_score(y_test, y, average = 'micro'),
                recall_score(y_test, y, average = 'micro')
])
    return res

def print_table(res):
    # Compute mean and std
    final = {}
    for model in res:
        arr = np.array(res[model])
        final[model] = {
            "Name" :model,
            "time" : arr[:, 0].mean().round(2),
            "f1_score": [arr[:,1].mean().round(5),arr[:,1].std().round(5)],
            "Precision": arr[: 2].mean().round(5).round(5),
            "Recall": arr[: 3].mean().round(5).round(5)
                    }

    df = pd.DataFrame.from_dict(final, orient="index").round(3)
    return df

In [5]:
# df0_pre = pd.read_csv('../data/emotion_final.csv')

# exclude = set(string.punctuation) # exclude = punctuation strings
# stop_word = stopwords.words('english') # we choosing stop words of english dict
# stop_word_punct = stop_word.extend(exclude) # we add strings punctions to stop word dict

# df0_pre.tokenized_sents = df0_pre.apply(lambda row: word_tokenize(row['Text']), axis=1) # Tokenization
# df0_pre.tokenized_sents = df0_pre.tokenized_sents.apply(lambda x: [item for item in x if item not in stop_word_punct])
# df0_pre.tokenized_sents = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df0_pre.tokenized_sents] 
# df0_pre.tokenized_sents = df0_pre.tokenized_sents.apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
# df0_pre.tokenized_sents = df0_pre.tokenized_sents.apply(lambda x: [porter.stem(y) for y in x]) # Stem every word.
# df0_pre.tokenized_sents = df0_pre.tokenized_sents.apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word.
# #delete list in list, **********need to be in one line*********
# dz = df0_pre.tokenized_sents
# dz = [[' '.join(i)][0] for i in dz]
# df0_pre.tokenized_sents = dz

In [6]:
# df1 = pd.read_csv('../data/kaggle_emotion.csv')

# exclude = set(string.punctuation) # exclude = punctuation strings
# stop_word = stopwords.words('english') # we choosing stop words of english dict
# stop_word_punct = stop_word.extend(exclude) # we add strings punctions to stop word dict

# corpus = np.array(df1["Text"])
# targets = np.array(df1["Emotion"])
# targets = np.array([1 if x == "sadness" else 2 if x=="anger" else 3 if x=="love" else 4 if x=="surprise" else 5 if x=="fear" 
# else 6 for x in targets])

In [7]:
df1 = pd.read_csv('../data/yes_emotion.csv')

exclude = set(string.punctuation) # exclude = punctuation strings
stop_word = stopwords.words('english') # we choosing stop words of english dict
stop_word_punct = stop_word.extend(exclude) # we add strings punctions to stop word dict

corpus = np.array(df1['content'])
targets = np.array(df1['sentiment'])
targets = np.array([1 if x == "sadness" else 2 if x =="anger" else 3 if x =="love" else 4 if x =="surprise" else 5 if x =="fear" else 6 if x == 'empty' else 7 if x =='enthusiasm' else 8 if x == 'neutral' else 9 if x == 'worry' else 10 if x == 'fun' else 11 if x == 'hate' else 12 if x == 'happiness' else 13 if x == 'boredom' else 14 for x in targets])



In [8]:
pipe0 = Pipeline([
    ('vect', CountVectorizer()),
    ('sgd', SGDClassifier()),
])
pipe1 = Pipeline([
    ('vect', CountVectorizer()),
    ('svm', SVC()),
])
pipe2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sgd', SGDClassifier()),
])
pipe3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm', LinearSVC()),
])
pipe4 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm', SVC()),
])
pipe5 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('logit', LogisticRegression()),
])
pipe6 = Pipeline([
    ('vect', CountVectorizer()),
    ('mult_nb', MultinomialNB()),
])
pipe7 = Pipeline([
    ('vect', CountVectorizer()),
    ('compl_nb', ComplementNB()),
])
pipe8 = Pipeline([
    ('vect', CountVectorizer()),
    ('bern_nb', BernoulliNB()),
])

In [9]:
# res = run_pipes([pipe0, pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7, pipe8], splits=5)
# oui2 = print_table(res)


In [70]:
oui2 = print_table(res)
oui2

,Name,time,f1_score,Precision,Recall
vect-sgd,vect-sgd,1.25,"[0.3218, 0.0022]",0.574,0.565
vect-svm,vect-svm,273.38,"[0.34968, 0.00266]",67.362,67.627
vect-tfidf-sgd,vect-tfidf-sgd,0.76,"[0.3184, 0.00366]",0.432,0.431
vect-tfidf-svm,vect-tfidf-svm,174.39,"[0.33335, 0.01733]",44.189,29.690
vect-tfidf-logit,vect-tfidf-logit,12.91,"[0.35065, 0.00427]",3.348,3.497
vect-mult_nb,vect-mult_nb,0.42,"[0.3076, 0.00188]",0.332,0.333
vect-compl_nb,vect-compl_nb,0.41,"[0.29007, 0.00385]",0.321,0.324
vect-bern_nb,vect-bern_nb,0.40,"[0.29905, 0.00356]",0.320,0.324


In [72]:

col_name = ['Name','Time/ms','F1 Score | Ecart','Precision','Recall']
# oui2.colmuns = col_name
oui2.rename(columns={'Name':'Name','time':'Time/ms','f1_score':'F1 Score | Ecart','Precision':'Precision','Recall':'Recall'}, inplace=True)
oui2.to_csv('yes_pre.csv')

In [65]:
non = pd.read_csv('yes_pre.csv')
non


,Name,time,f1_score,Precision,Recall
vect-sgd,vect-sgd,1.25,"[0.3218, 0.0022]",0.574,0.565
vect-svm,vect-svm,273.38,"[0.34968, 0.00266]",67.362,67.627
vect-tfidf-sgd,vect-tfidf-sgd,0.76,"[0.3184, 0.00366]",0.432,0.431
vect-tfidf-svm,vect-tfidf-svm,174.39,"[0.33335, 0.01733]",44.189,29.690
vect-tfidf-logit,vect-tfidf-logit,12.91,"[0.35065, 0.00427]",3.348,3.497
vect-mult_nb,vect-mult_nb,0.42,"[0.3076, 0.00188]",0.332,0.333
vect-compl_nb,vect-compl_nb,0.41,"[0.29007, 0.00385]",0.321,0.324
vect-bern_nb,vect-bern_nb,0.40,"[0.29905, 0.00356]",0.320,0.324


In [50]:
oui1          

,Unnamed: 0,Name,Time/ms,F1 Score | Ecart,Precision,Recall
0,0,vect-sgd,NaN,NaN,0.777,0.778
1,1,vect-svm,NaN,NaN,22.238,22.929
2,2,vect-tfidf-sgd,NaN,NaN,0.761,0.773
3,3,vect-tfidf-svm,NaN,NaN,11.863,8.183
4,4,vect-tfidf-logit,NaN,NaN,1.703,1.833
5,5,vect-mult_nb,NaN,NaN,0.627,0.624
6,6,vect-compl_nb,NaN,NaN,0.720,0.718
7,7,vect-bern_nb,NaN,NaN,0.557,0.552


In [33]:
# oui2.to_csv('yes_prediction_pipe.csv',index=False)
col_name = ['Name','Time/ms','F1 Score | Ecart','Precision','Recall']
yes = pd.read_csv('yes_prediction_pipe.csv',columns = col_name)

yes.columns = col_name
# yes.to_csv('yes_pre.csv')
yes

TypeError: parser_f() got an unexpected keyword argument 'columns'

In [27]:
oui1 = pd.read_csv('../data/kaggle_pre.csv')
oui1

,Unnamed: 0,Name,Time/ms,F1 Score | Ecart,Precision,Recall
0,0,vect-sgd,0.50,"[0.88658, 0.00343]",0.777,0.778
1,1,vect-svm,93.38,"[0.78723, 0.00312]",22.238,22.929
2,2,vect-tfidf-sgd,0.43,"[0.88998, 0.00316]",0.761,0.773
3,3,vect-tfidf-svm,48.62,"[0.86768, 0.02397]",11.863,8.183
4,4,vect-tfidf-logit,5.09,"[0.85391, 0.00548]",1.703,1.833
5,5,vect-mult_nb,0.27,"[0.74534, 0.00501]",0.627,0.624
6,6,vect-compl_nb,0.28,"[0.87707, 0.00286]",0.720,0.718
7,7,vect-bern_nb,0.28,"[0.65522, 0.00809]",0.557,0.552


In [12]:
# oui = pd.DataFrame(data=oui2)
# col_name = ['Name','Time/ms','F1 Score | Ecart','Precision','Recall']
# oui.columns = col_name
# oui.to_csv('prediction_pipe.csv')
# oui2
oui2.to_csv('prediction_pipe.csv',index=False)
oui3 = pd.read_csv('prediction_pipe.csv')

.columns

SyntaxError: invalid syntax (<ipython-input-12-52da46618a2f>, line 9)

In [9]:
pipe0 = Pipeline([
    ('vect', CountVectorizer()),
    ('sgd', SGDClassifier()),
])
pipe1 = Pipeline([
    ('vect', CountVectorizer()),
    ('svm', SVC()),
])
pipe2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('sgd', SGDClassifier()),
])
pipe3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm', LinearSVC()),
])
pipe4 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svm', SVC()),
])
pipe5 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('logit', LogisticRegression()),
])
pipe6 = Pipeline([
    ('vect', CountVectorizer()),
    ('mult_nb', MultinomialNB()),
])
pipe7 = Pipeline([
    ('vect', CountVectorizer()),
    ('compl_nb', ComplementNB()),
])
pipe8 = Pipeline([
    ('vect', CountVectorizer()),
    ('bern_nb', BernoulliNB()),
])

In [10]:
# run base pipes
# res = run_pipes([pipe0, pipe1, pipe2, pipe3, pipe4, pipe5, pipe6, pipe7, pipe8], splits=5)
# print_table(res)

# 	time	f1_score
# vect-sgd	0.64	[0.887, 0.005]
# vect-svm	91.26	[0.787, 0.003]
# vect-tfidf-sgd	0.50	[0.89, 0.003]
# vect-tfidf-svml	0.59	[0.891, 0.007]
# vect-tfidf-svm	97.59	[0.845, 0.006]
# vect-tfidf-logit	4.45	[0.854, 0.006]
# vect-mult_nb	0.25	[0.745, 0.005]
# vect-compl_nb	0.24	[0.877, 0.003]
# vect-bern_nb	0.25	[0.655, 0.008]

In [11]:
df1 = pd.read_csv('../data/emotion_final.csv')

# print(df1['Emotion'].unique())
df_count_emotion = df1.groupby(['Emotion']).size().reset_index(name='Count')
print(df_count_emotion) 

    Emotion  Count
0     anger   2993
1      fear   2652
2     happy   7029
3      love   1641
4   sadness   6265
5  surprise    879


In [12]:
exclude = set(string.punctuation) # exclude = punctuation strings
stop_words = stopwords.words('english') # we choosing stop words of english dict
stop_words.extend(exclude) # we add strings punctions to stop word dict
lemma = WordNetLemmatizer()
stemmer = SnowballStemmer("english") # we choosing the language english for the stemmization 
porter = PorterStemmer() 
lancaster=LancasterStemmer()


In [25]:
df_oui = pd.read_csv('../data/emotion_final.csv')
df_oui

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [24]:
df_pre = pd.read_csv('../data/emotion_final.csv')
df_pre['Text'] = df_pre.apply(lambda row: word_tokenize(row['Text']), axis=1) # Tokenization
df_pre['Text'] = df_pre['Text'].apply(lambda x: [item for item in x if item not in stop_words]) # Stop wordization :) coucou anne-laure
df_pre['Text'] = [[lemma.lemmatize(word) for word in each if word not in stop_word] for each in df_pre['Text']]  # Lemmization
# df_pre['Text'] = df1['Text'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word. with snowball('english')
# df_pre['Text'] = df1['Text'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word. with porter
# df_pre['Text'] = df1['Text'].apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word. with lancaster
dz = df_pre['Text']
dz = [[' '.join(i)][0] for i in dz] 
df_pre['Text'] = dz
df_pre
# df_pre.to_csv('./data/emotion_final_pre.csv')

,Text,Emotion
0,didnt feel humiliated,sadness
1,go feeling hopeless damned hopeful around some...,sadness
2,im grabbing minute post feel greedy wrong,anger
3,ever feeling nostalgic fireplace know still pr...,love
4,feeling grouchy,anger
...,...,...
21454,Melissa stared friend dism,fear
21455,Successive state election seen governing party...,fear
21456,Vincent irritated dismay,fear
21457,Kendall-Hume turned back face dismayed coup,fear


In [23]:
# df1 = pd.read_csv('../data/text_emotion.csv')
# df1['content'] = df1.apply(lambda row: word_tokenize(row['content']), axis=1) # Tokenization
# df1['content'] = df1['content'].apply(lambda x: [item for item in x if item not in stop_word]) # Stop wordization :) coucou anne-laure
# # df1['content'] = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df1['Text']]  # Lemmization
# # df1['content'] = df1['content'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word. with snowball('english')
# df1['content'] = df1['content'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word. with porter
# df1['content'] = df1['content'].apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word. with lancaster

# dz = df1['content']
# dz = [[' '.join(i)][0] for i in dz] 

# df1['content'] = dz
df1.to_csv('text_emotion_pre.csv')
df1

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,tiffanylu know listenin bad habit ear start fr...
1,1956967666,sadness,wannamama,layin n bed headach ughhhh ... waitin cal ...
2,1956967696,sadness,coolfunky,fun ceremon ... gloom friday ...
3,1956967789,enthusiasm,czareaquino,want hang friend soon
4,1956968416,neutral,xkilljoyx,dannycastillo we want trad someon houston tick...
...,...,...,...,...
39995,1753918954,neutral,showMe_Heaven,johnlloydtayl
39996,1753919001,love,drapeaux,happ moth day al lov
39997,1753919005,love,JenniRox,happ moth 's day momm wom man long 're 'momma ...
39998,1753919043,happiness,ipdaman1,niariley wassup beaut follow me peep out my ne...


In [ ]:
data = {'Emotion':df1['Emotion'],'Text':df1['Text']}
df28 = pd.DataFrame(data=data)
df28

In [55]:
df1 = pd.read_csv('../data/emotion_final.csv')
df1['Text'] = df1.apply(lambda row: word_tokenize(row['Text']), axis=1) # Tokenization
df1['Text'] = df1['Text'].apply(lambda x: [item for item in x if item not in stop_sw]) # Stop wordization :) coucou anne-laure
# df1['Text'] = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df1['Text']]  # Lemmization
df1['Text'] = df1['Text'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word. with snowball('english')
df1['Text'] = df1['Text'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word. with porter
df1['Text'] = df1['Text'].apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word. with lancaster

dz = df1['Text']
dz = [[' '.join(i)][0] for i in dz] 

df1['Text'] = dz

df1

NameError: name 'stop_sw' is not defined

In [ ]:
df2 = pd.read_csv('../data/text_emotion.csv')

df2['tokenized_sents'] = df2.apply(lambda row: word_tokenize(row['content']), axis=1) # Tokenization
df2['tokenized_sents'] = df2['tokenized_sents'].apply(lambda x: [item for item in x if item not in stop_sw])
df2.tokenized_sents = [[lemma.lemmatize(word) for word in each if word not in stop_sw] for each in df2.tokenized_sents] 
df2['tokenized_sents'] = df2['tokenized_sents'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
df2['tokenized_sents'] = df2['tokenized_sents'].apply(lambda x: [porter.stem(y) for y in x]) # Stem every word.
df2['tokenized_sents'] = df2['tokenized_sents'].apply(lambda x: [lancaster.stem(y) for y in x]) # Stem every word.

df2

In [ ]:
corpus = df2['tokenized_sents']
x = corpus
x = [[' '.join(i)][0] for i in x]

df2['tokenized_sents'] = x
df2

In [58]:
corpus = np.array(df1['Text'])
x = corpus

# TF-IDF Vectorizer 
v_tf = TfidfVectorizer(stop_words=stop_sw, tokenizer=tokenize, ngram_range=(1,2))
x_tf = v_tf.fit_transform(x)
# TF-IDF Model
idf_values = dict(zip(v_tf.get_feature_names(), v_tf.idf_))
words_name = v_tf.get_feature_names()

print("-- TF-IDF vector shape :",x_tf.shape)

NameError: name 'stop_sw' is not defined

In [ ]:
corpus = np.array(df1['Text'])
x = corpus

cv = CountVectorizer(stop_words=stop_sw, tokenizer=tokenize, ngram_range=(1,2))
x_cv = cv.fit_transform(x) ## Count Vectorizer Model
print("-- Count_Vectorizer shape :",x_cv.shape)

In [ ]:
# Train pack with TF-IDF Vectorizer 
x_tf_train, x_tf_test, y_train, y_test = train_test_split(x_tf, y, test_size=0.20, random_state=0)
print('-- Training pack TF-IDF:',"\n")
print('-- x_tf_train rows :',np.size(x_tf_train))
print('-- x_tf_test rows :',np.size(x_tf_test))
print('-- ratio in % :', round(np.size(x_tf_test)/(np.size(x_tf_train)+np.size(x_tf_test))*100,2),"\n") 


In [ ]:
corpus = np.array(df2['content'])
x = corpus

# TF-IDF Vectorizer 
v_tf = TfidfVectorizer(stop_words=stop_sw, tokenizer=tokenize, ngram_range=(1,2))
x_tf = v_tf.fit_transform(x)
# TF-IDF Model
idf_values = dict(zip(v_tf.get_feature_names(), v_tf.idf_))
words_name = v_tf.get_feature_names()

print("-- TF-IDF vector shape :",x_tf.shape)

In [ ]:
corpus = np.array(df2['content'])
x = corpus

cv = CountVectorizer(stop_words=stop_sw, tokenizer=tokenize, ngram_range=(1,2))
x_cv = cv.fit_transform(x) ## Count Vectorizer Model
print("-- Count_Vectorizer shape :",x_cv.shape)

In [66]:
df1 = pd.read_csv('../data/emotion_final.csv')

x = df1.Text
y = df1.Emotion

freq_top = get_top_n_words(x,"up",100)


df_up = pd.DataFrame(freq_top, columns =['Word','Number of times'])
y_nbr = df_up['Number of times']
x_word = df_up['Word']
# df_down = pd.DataFrame(freq_down, columns =['Word','Number of times'])


In [67]:


# freq = subsample(wrank)
# r = np.arange(len(freq))

trace = go.Bar(
                x = x_word.head(30),
                y = y_nbr,
                name = "Le score universitaire pour le transfert de connaissances par pays",
                marker = dict(color = 'rgba(255, 87, 51, 0.5)', line = dict(color ='rgb(0,0,0)',width =2.5)),
                text = df_up['Word'])

layout = go.Layout(barmode = "group",
                  title = 'Fréquence d’apparition des mots ',
                  yaxis = dict(title = 'word frequency'),
                  xaxis = dict(title = 'word rank'))
fig = go.Figure(data = trace, layout = layout)
iplot(fig)

In [94]:

dftableau = df1
ParGroup = dftableau.groupby('Emotion')
y1=ParGroup['Emotion'].size().sort_values(ascending=False)


trace = go.Bar(
            x = y1.index.get_level_values(0).tolist(),                  
            y = y1,
            name = 'Income',
            type = 'bar',
                )

layout = go.Layout(
            xaxis = {'title': 'Emotions'},
            yaxis = {'title': 'Occurence'},
            barmode = 'relative',
            title = 'Emotions ranked by occurence in Emotion_final')

fig = go.Figure(data = trace, layout = layout)

iplot(fig)

In [112]:
df1 = pd.read_csv('../data/emotion_final.csv')

x = df1.groupby('Emotion')

trace = go.Bar(
            x = x,                  
            y = x,
            name = 'Income',
            type = 'bar',
                )

layout = go.Layout(
            xaxis = {'title': 'Emotions'},
            yaxis = {'title': 'Occurence'},
            barmode = 'relative',
            title = 'Emotions ranked by occurence in Emotion_final')

fig = go.Figure(data = trace, layout = layout)

iplot(fig)

ValueError: 
    Invalid value of type 'pandas.core.groupby.generic.DataFrameGroupBy' received for the 'x' property of bar
        Received value: <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f155f419c90>

    The 'x' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

In [ ]:

df1 = pd.read_csv('../data/kaggle_emotion.csv')


emotion_hist = go.Figure(px.histogram(df1, x="Emotion", color= "Emotion", title="Histogramme Emotion").update_xaxes(categoryorder ="total descending"))
emotion_hist.update_layout(
    paper_bgcolor='rgba(0,0,0,0.65)',
    plot_bgcolor='rgba(0,0,0,0.65)')


fig.show()

In [130]:
df1 = pd.read_csv('../data/emotion_final.csv')


emotion_hist = go.Figure(px.histogram(df1, x="Emotion", color= "Emotion", title="Histogramme Emotion").update_xaxes(categoryorder ="total descending"))
emotion_hist.update_layout(
    paper_bgcolor='rgba(0,0,0,0.65)',
    plot_bgcolor='rgba(0,0,0,0.65)')


fig.show()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'x' property of histogram
        Received value: 'Emotion'

    The 'x' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series